Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.004695
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.251878
Minibatch accuracy: 56.2%
Validation accuracy: 60.1%
Minibatch loss at step 100: 0.919124
Minibatch accuracy: 81.2%
Validation accuracy: 74.4%
Minibatch loss at step 150: 1.646754
Minibatch accuracy: 56.2%
Validation accuracy: 69.8%
Minibatch loss at step 200: 0.634265
Minibatch accuracy: 81.2%
Validation accuracy: 75.7%
Minibatch loss at step 250: 1.069804
Minibatch accuracy: 68.8%
Validation accuracy: 75.4%
Minibatch loss at step 300: 0.900661
Minibatch accuracy: 68.8%
Validation accuracy: 79.8%
Minibatch loss at step 350: 0.728580
Minibatch accuracy: 75.0%
Validation accuracy: 80.5%
Minibatch loss at step 400: 0.781217
Minibatch accuracy: 75.0%
Validation accuracy: 80.6%
Minibatch loss at step 450: 1.172858
Minibatch accuracy: 75.0%
Validation accuracy: 80.5%
Minibatch loss at step 500: 0.669522
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1,1,1,1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(hidden1, [1,2,2,1], [1,2,2,1], padding='SAME')
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1,1,1,1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 = tf.nn.max_pool(hidden2, [1,2,2,1], [1,2,2,1], padding='SAME')
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [16]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.728092
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.278092
Minibatch accuracy: 50.0%
Validation accuracy: 50.7%
Minibatch loss at step 100: 0.963399
Minibatch accuracy: 68.8%
Validation accuracy: 67.5%
Minibatch loss at step 150: 1.372500
Minibatch accuracy: 62.5%
Validation accuracy: 65.2%
Minibatch loss at step 200: 0.706740
Minibatch accuracy: 81.2%
Validation accuracy: 76.5%
Minibatch loss at step 250: 0.891829
Minibatch accuracy: 68.8%
Validation accuracy: 70.9%
Minibatch loss at step 300: 0.817115
Minibatch accuracy: 68.8%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.849051
Minibatch accuracy: 75.0%
Validation accuracy: 81.2%
Minibatch loss at step 400: 0.775852
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 450: 1.059168
Minibatch accuracy: 81.2%
Validation accuracy: 81.3%
Minibatch loss at step 500: 0.586295
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Mi

Validation accuracy: 87.7%
Minibatch loss at step 4550: 0.294526
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 4600: 0.310650
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 4650: 0.562676
Minibatch accuracy: 87.5%
Validation accuracy: 88.1%
Minibatch loss at step 4700: 0.170023
Minibatch accuracy: 93.8%
Validation accuracy: 88.1%
Minibatch loss at step 4750: 0.096869
Minibatch accuracy: 93.8%
Validation accuracy: 88.2%
Minibatch loss at step 4800: 0.303183
Minibatch accuracy: 93.8%
Validation accuracy: 88.1%
Minibatch loss at step 4850: 0.418152
Minibatch accuracy: 81.2%
Validation accuracy: 88.2%
Minibatch loss at step 4900: 0.391669
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 4950: 0.319613
Minibatch accuracy: 93.8%
Validation accuracy: 88.1%
Minibatch loss at step 5000: 0.236477
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Minibatch loss at step 5050: 0.394442
Minibatch accuracy: 81.2%

Validation accuracy: 88.1%
Minibatch loss at step 9050: 0.417606
Minibatch accuracy: 87.5%
Validation accuracy: 88.6%
Minibatch loss at step 9100: 0.624536
Minibatch accuracy: 81.2%
Validation accuracy: 88.9%
Minibatch loss at step 9150: 0.477987
Minibatch accuracy: 87.5%
Validation accuracy: 88.4%
Minibatch loss at step 9200: 0.671596
Minibatch accuracy: 75.0%
Validation accuracy: 89.2%
Minibatch loss at step 9250: 0.103736
Minibatch accuracy: 100.0%
Validation accuracy: 89.6%
Minibatch loss at step 9300: 1.089878
Minibatch accuracy: 75.0%
Validation accuracy: 89.0%
Minibatch loss at step 9350: 0.258604
Minibatch accuracy: 93.8%
Validation accuracy: 88.9%
Minibatch loss at step 9400: 0.388195
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 9450: 0.348145
Minibatch accuracy: 87.5%
Validation accuracy: 88.9%
Minibatch loss at step 9500: 0.407734
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 9550: 0.089953
Minibatch accuracy: 100.

Minibatch loss at step 13500: 0.236755
Minibatch accuracy: 93.8%
Validation accuracy: 89.8%
Minibatch loss at step 13550: 0.249092
Minibatch accuracy: 87.5%
Validation accuracy: 90.0%
Minibatch loss at step 13600: 0.274053
Minibatch accuracy: 87.5%
Validation accuracy: 89.7%
Minibatch loss at step 13650: 0.004652
Minibatch accuracy: 100.0%
Validation accuracy: 89.5%
Minibatch loss at step 13700: 0.699845
Minibatch accuracy: 81.2%
Validation accuracy: 89.7%
Minibatch loss at step 13750: 0.236869
Minibatch accuracy: 93.8%
Validation accuracy: 89.6%
Minibatch loss at step 13800: 0.139147
Minibatch accuracy: 93.8%
Validation accuracy: 89.8%
Minibatch loss at step 13850: 0.299168
Minibatch accuracy: 93.8%
Validation accuracy: 89.6%
Minibatch loss at step 13900: 0.057898
Minibatch accuracy: 100.0%
Validation accuracy: 89.5%
Minibatch loss at step 13950: 0.203205
Minibatch accuracy: 93.8%
Validation accuracy: 89.6%
Minibatch loss at step 14000: 0.152193
Minibatch accuracy: 100.0%
Validation a

Minibatch loss at step 17950: 0.197769
Minibatch accuracy: 100.0%
Validation accuracy: 90.0%
Minibatch loss at step 18000: 0.362877
Minibatch accuracy: 93.8%
Validation accuracy: 89.7%
Minibatch loss at step 18050: 0.074393
Minibatch accuracy: 100.0%
Validation accuracy: 90.0%
Minibatch loss at step 18100: 0.620059
Minibatch accuracy: 81.2%
Validation accuracy: 90.0%
Minibatch loss at step 18150: 0.107735
Minibatch accuracy: 93.8%
Validation accuracy: 89.8%
Minibatch loss at step 18200: 0.185200
Minibatch accuracy: 93.8%
Validation accuracy: 90.1%
Minibatch loss at step 18250: 0.241314
Minibatch accuracy: 87.5%
Validation accuracy: 90.0%
Minibatch loss at step 18300: 0.409318
Minibatch accuracy: 81.2%
Validation accuracy: 89.9%
Minibatch loss at step 18350: 0.329045
Minibatch accuracy: 87.5%
Validation accuracy: 90.0%
Minibatch loss at step 18400: 0.195821
Minibatch accuracy: 93.8%
Validation accuracy: 90.1%
Minibatch loss at step 18450: 0.086273
Minibatch accuracy: 93.8%
Validation ac

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [34]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
reg_val = 1e-4
drop_out = 0.5

graph = tf.Graph()

with graph.as_default():

  # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
    layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
    def model(data, keep_prob):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
        bias1 = tf.nn.relu(conv1 + layer1_biases)

        pool1 = tf.nn.avg_pool(bias1, [1,2,2,1], [1,2,2,1], padding='VALID')
        
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1,1,1,1], padding='VALID')
        bias2 = tf.nn.relu(conv2 + layer2_biases)

        pool2 = tf.nn.avg_pool(bias2, [1,2,2,1], [1,2,2,1], padding='VALID')
        
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        
        hidden1 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        drop1 = tf.nn.dropout(hidden1, keep_prob)
        
        hidden2 = tf.nn.relu(tf.matmul(hidden1, layer4_weights) + layer4_biases)
        drop2 = tf.nn.dropout(hidden2, keep_prob)
        
        return tf.matmul(drop2, layer5_weights) + layer5_biases

    # Training computation.
    logits = model(tf_train_dataset, drop_out)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.85, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, drop_out))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, drop_out))

In [36]:
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in xrange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.749097
Minibatch accuracy: 12.5%
Validation accuracy: 10.1%
Minibatch loss at step 100: 1.389420
Minibatch accuracy: 62.5%
Validation accuracy: 34.2%
Minibatch loss at step 200: 1.435745
Minibatch accuracy: 37.5%
Validation accuracy: 51.6%
Minibatch loss at step 300: 1.381861
Minibatch accuracy: 50.0%
Validation accuracy: 59.9%
Minibatch loss at step 400: 1.424675
Minibatch accuracy: 68.8%
Validation accuracy: 63.9%
Minibatch loss at step 500: 1.014989
Minibatch accuracy: 68.8%
Validation accuracy: 67.8%
Minibatch loss at step 600: 0.712527
Minibatch accuracy: 87.5%
Validation accuracy: 71.1%
Minibatch loss at step 700: 1.222455
Minibatch accuracy: 68.8%
Validation accuracy: 74.3%
Minibatch loss at step 800: 0.781028
Minibatch accuracy: 75.0%
Validation accuracy: 74.2%
Minibatch loss at step 900: 1.207660
Minibatch accuracy: 62.5%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 0.564742
Minibatch accuracy: 87.5%
Validation accuracy: 74.7%

Validation accuracy: 85.4%
Minibatch loss at step 9100: 0.491643
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 9200: 0.654703
Minibatch accuracy: 75.0%
Validation accuracy: 86.0%
Minibatch loss at step 9300: 0.969495
Minibatch accuracy: 75.0%
Validation accuracy: 85.7%
Minibatch loss at step 9400: 0.293799
Minibatch accuracy: 93.8%
Validation accuracy: 86.0%
Minibatch loss at step 9500: 0.571058
Minibatch accuracy: 81.2%
Validation accuracy: 85.7%
Minibatch loss at step 9600: 0.421944
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 9700: 0.788757
Minibatch accuracy: 68.8%
Validation accuracy: 85.7%
Minibatch loss at step 9800: 0.376735
Minibatch accuracy: 93.8%
Validation accuracy: 85.5%
Minibatch loss at step 9900: 0.216017
Minibatch accuracy: 93.8%
Validation accuracy: 85.8%
Minibatch loss at step 10000: 0.496761
Minibatch accuracy: 93.8%
Validation accuracy: 85.9%
Minibatch loss at step 10100: 0.305303
Minibatch accuracy: 87.

Minibatch loss at step 18000: 0.487349
Minibatch accuracy: 81.2%
Validation accuracy: 86.5%
Minibatch loss at step 18100: 0.939832
Minibatch accuracy: 75.0%
Validation accuracy: 86.5%
Minibatch loss at step 18200: 0.341129
Minibatch accuracy: 81.2%
Validation accuracy: 86.5%
Minibatch loss at step 18300: 0.599357
Minibatch accuracy: 75.0%
Validation accuracy: 86.7%
Minibatch loss at step 18400: 0.328521
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 18500: 0.393354
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 18600: 0.155260
Minibatch accuracy: 100.0%
Validation accuracy: 86.6%
Minibatch loss at step 18700: 0.536758
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 18800: 0.517509
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 18900: 0.288579
Minibatch accuracy: 93.8%
Validation accuracy: 86.5%
Minibatch loss at step 19000: 0.107541
Minibatch accuracy: 100.0%
Validation ac

__Conclusion__: While we originally believed that introducing dropout will help in improving accuracy, in reality however it has only worsened. This is because adding dropout adds up a new hyper-parameter, which serves us well only when we tune it well.<br>
To increase performance, we can increasd the depth and make it different between the layers.